In [1]:
!wget https://www.dropbox.com/s/xtc56u0npo3ou04/cows_harmonic.t7 -O cows_harmonic.t7

--2019-04-10 11:48:26--  https://www.dropbox.com/s/xtc56u0npo3ou04/cows_harmonic.t7
Resolving www.dropbox.com (www.dropbox.com)... 162.125.70.1, 2620:100:6026:1::a27d:4601
Connecting to www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/xtc56u0npo3ou04/cows_harmonic.t7 [following]
--2019-04-10 11:48:32--  https://www.dropbox.com/s/raw/xtc56u0npo3ou04/cows_harmonic.t7
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucbc561ca371bcac5ca3e6e53523.dl.dropboxusercontent.com/cd/0/inline/AesqEAQr5vLJAoRkaZI9do0tI1gFu40IR__YYNN3V21qgfYQvcGbBKFJiWfn9MQX8s_09F8lQzuswt9ae6V0jCrwnfdURmDR3BGOYcMMLLNARw/file# [following]
--2019-04-10 11:48:32--  https://ucbc561ca371bcac5ca3e6e53523.dl.dropboxusercontent.com/cd/0/inline/AesqEAQr5vLJAoRkaZI9do0tI1gFu40IR__YYNN3V21qgfYQvcGbBKFJiWfn9MQX8s_09F8lQzuswt9ae6V0jCrwnfdURmDR3BGOYcMMLLNARw/file
Reso

In [1]:
import torch 
from torchvision import transforms
from harmonic import cluster
from models import PSPNet

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose(
    [transforms.RandomCrop(512),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                          std=[0.229, 0.224, 0.225])])

sins = [[23.50503921508789, 0.0, 1.2836638689041138],
 [-9.183862686157227, 0.0, 0.6610772013664246],
 [9.274807929992676, 0.0, 0.4241010546684265],
 [46.33891677856445, 0.0, 0.14550864696502686],
 [-5.840827465057373, 0.0, 0.052969418466091156],
 [-39.868263244628906, 0.0, 0.7093945741653442],
 [0.0, -12.252435684204102, -11.785104751586914],
 [0.0, 9.418264389038086, -3.8895671367645264],
 [0.0, 6.086294651031494, 7.134507656097412],
 [0.0, 80.29950714111328, 7.632736682891846],
 [0.0, 8.219144821166992, 4.2953972816467285],
 [0.0, -30.652725219726562, -6.30106258392334]]


net = torch.nn.DataParallel(
    PSPNet(sins=sins,pretrained=False,extended=True).to(device), [0]).to(device)

wdict = torch.load('cows_harmonic.t7')
for k in wdict.keys():
    if k.endswith('num_batches_tracked'):
        wdict.pop(k)
net.load_state_dict(wdict)
net.eval()
print("Model loaded")

Model loaded


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

img = Image.open("images/cow.jpg").convert('RGB')
#estimate the embeddings
x = transform(img)
emb = net(x.unsqueeze(0).to(device)).detach().cpu().numpy()
#cluster embeddings space
cc = cluster(emb[0],0.5,minimal_area=1000)

f,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,10))
ax[0].imshow(x.numpy().transpose(1,2,0)*[0.229, 0.224, 0.225]+[0.485, 0.456, 0.406])
ax[1].imshow(cc)